In [1]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
ds = pd.read_csv('https://raw.githubusercontent.com/LuisSante/Datasets/main/app_reviews.csv')
ds

,package_name,review,date,star
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5
...,...,...,...,...
288060,com.termux.api,it doesn't do anything after installing this i...,June 24 2016,3
288061,com.termux.api,I like this app . Its is very helpful for use....,June 20 2016,5
288062,com.termux.api,Finally Brings back the Unix command line to A...,May 20 2016,5
288063,com.termux.api,The API feature is great just need loads more...,May 05 2016,5


In [9]:
list_ = []  
for i in range(len(ds['package_name'].unique())):
    ds_aux = ds.loc[ds['package_name'] == ds['package_name'].unique()[i]]
    list_.append({'package_name':ds['package_name'].unique()[i], 'size': len(ds_aux)})

list_ = sorted(list_, key=lambda x: x['size'], reverse=True)
df = ds[ds['package_name'] == list_[40]['package_name']]
corpus = list(df['review'])

ds_new = pd.DataFrame(corpus)
print(ds_new)

                                                      0
0     Amazing Its great but can someone explain to m...
1     Seems this is one of the best app although I d...
2     Great; needs material design UI. Please use ma...
3                                             Great app
4                                          Cathy Female
...                                                 ...
1090  First time experimenting with this theorpy in ...
1091  Tried and true...no frills without the bells a...
1092                            One of the best so far.
1093              I quit smoking with the self hypnosis
1094   Helps me focus on work with kids running around!

[1095 rows x 1 columns]


In [12]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

In [13]:
clustering_model = AgglomerativeClustering(n_clusters=3, affinity="cosine" , linkage="complete") #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [14]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  3
['Amazing Its great but can someone explain to me how the hipnosis one works?', 'Cathy Female', 'I use this app and me wake up 7.00 am!!sometime I wake up 11.00am now I use this app I wake up morning', 'Love the app thank you I now have full night rest and not wake up multiple times every night.', "A good one  but... I used the Morphine pain killer and was a bit surprised to see a green color on the screen. Yes  we all know that green is the color of healing  but actually it stimulates the reproduction of all possible cells which bind our organism. That means the bad tumorous cells as well  in case we have cancer. That's why it is not a good color choice for that group of people. And who uses morphine more often than them? But leaving that aside  this is a great app!", 'Greay T', 'Powerfull limitless Good way of interact with out deep state of mind...thanks for make this easy  we will have lots of years together in many different way of relationship', 'Flaw sw code Im not su